# Rearranger

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
from functools import partial, wraps

In [ ]:
root = tk.Tk()
root.withdraw()

# Open dialog to choose the .txt file
file_type = [('Text File', '*.txt')]
default_dir = 'Input'
file_path = filedialog.askopenfilename(filetypes=file_type, initialdir=default_dir)

# Read the .txt file into a dataframe
raw_df = pd.read_csv(file_path, sep = "\t")

raw_df.head()


In [ ]:
# Remove unnecessary columns
for col in raw_df.columns:
    if "unnamed" in col.lower() or "prob" in col.lower():
        raw_df.drop(col, axis=1, inplace=True)

raw_df

In [ ]:
# Extract the first frame of the video to get a sample image

# Open dialog to choose the .mp4 file
file_type = [('Video File', '*.mp4')]
default_dir = 'Input'
file_path = filedialog.askopenfilename(filetypes=file_type, initialdir=default_dir)
video_P = Path(file_path)

cap = cv2.VideoCapture(file_path)
_, frame = cap.read()
cap.release()

# Export frame as a .png file
image_P = video_P.parent / (video_P.stem + ".png")
cv2.imwrite(str(image_P), frame)


In [ ]:
def get_tanks(im):
    example_tanks = {}

    tank_names = ['Top Left', 'Top Right', 'Bottom Left']

    def tank_selector(im, tank_name):
        while True:
            selected_pixel = cv2.selectROI('Selecting tank at position'+str(tank_name), im)
            
            # print out the coordinates of top left and bottom right pixel of the ROI
            print('Tank at position: ', tank_name)
            print('Top left pixel: x = {}, y = {}'.format(selected_pixel[0], selected_pixel[1]))
            print('Bottom right pixel: x = {}, y = {}'.format(selected_pixel[0] + selected_pixel[2], selected_pixel[1] + selected_pixel[3]))
            print('Middle line: y = {}'.format(selected_pixel[1] + selected_pixel[3] / 2))
            tank_d = math.ceil((selected_pixel[2] + selected_pixel[3]) / 2)
            print('Diameter of the tank : ', tank_d)
            print()
            example_tanks[tank_name] = selected_pixel
            cv2.destroyAllWindows()
            break

    for name in tank_names:
        tank_selector(im, name)

    return example_tanks

In [ ]:
image_P = Path(r"C:\Code\ChungyuanProjects\TowerAssayAnalyzer\Input\Zebrafish Tower\A - Novel Tank Test\01 - Control (1st Batch).png")


In [ ]:

im = cv2.imread(str(image_P))
example_wells = get_tanks(im)

In [ ]:
tanks_col = 5
tanks_row = 2

# tanks number
# 10 9 8 7 6
# 5 4 3 2 1

tanks_dict = {}

TL_num = tanks_col * tanks_row
TR_num = TL_num - tanks_col + 1
BL_num = tanks_col

tanks_dict[TL_num] = example_wells['Top Left']
tanks_dict[TR_num] = example_wells['Top Right']
tanks_dict[BL_num] = example_wells['Bottom Left']
tanks_dict[1] = (tanks_dict[6][0] - (tanks_dict[10][0] - tanks_dict[5][0]), tanks_dict[6][1] - (tanks_dict[10][1] - tanks_dict[5][1]), tanks_dict[6][2]*(tanks_dict[5][2]/tanks_dict[10][2]), tanks_dict[6][3]*(tanks_dict[5][3]/tanks_dict[10][3]))

def fill_row(tanks_dict, start, end, display = True):
    x_sep = (tanks_dict[start][0] - tanks_dict[end][0])/(end - start)
    y_sep = (tanks_dict[start][1] - tanks_dict[end][1])/(end - start)
    w_sep = (tanks_dict[start][2] - tanks_dict[end][2])/(end - start)
    h_sep = (tanks_dict[start][3] - tanks_dict[end][3])/(end - start)

    for i in range(start+1, end):
        tanks_dict[i] = (tanks_dict[end][0] + x_sep*(end-i), tanks_dict[end][1] + y_sep*(end-i), tanks_dict[end][2] + w_sep*(end-i), tanks_dict[end][3] + h_sep*(end-i))
    
    if display:
        for key, value in locals().items():
            print(key + ' = ' + str(value))

    return tanks_dict

for row in range(0, tanks_row):
    tanks_dict = fill_row(tanks_dict, 1 + row*tanks_col, (row+1)*tanks_col)
    # row = 0, fill(tanks_dict, 1 + 0*5,  (0+1)*5)
    # row = 1, fill(tanks_dict, 1 + 1*5,  (1+1)*5)

# tanks_dict = fill_row(tanks_dict, 1, 5)
# tanks_dict = fill_row(tanks_dict, 6, 10)

# round up all the values in tanks_dict to integers
for key in tanks_dict:
    tanks_dict[key] = tuple([int(round(i)) for i in tanks_dict[key]])

tanks_dict


In [ ]:
im = cv2.imread(str(image_P))

# Display predicted tanks
def display_tanks():
    for i in range(1, 11):
        x, y, w, h = tanks_dict[i]
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(im, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Tanks', im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

display_tanks()


In [ ]:
import random
from Libs.misc import display_coords

def coord_generator(box, points = 1000):
    x, y, w, h = box
    coords = []
    for i in range(points):
        x_coord = random.randint(x, x+w)
        y_coord = random.randint(y, y+h)
        coords.append((x_coord, y_coord))
    return coords

def mix_up(input_dict, mix_ratio = 0.3):
    # input_dict has 10 keys 
    # keys are 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
    # values are coordinates inside the tanks
    # mix_ratio is the ratio of mixing up the coordinates
    # mix_ratio = 0.3 means 30% of the coordinates will be mixed up

    for i in range(1, len(input_dict)+1):
        if i == 1:
            continue
        else:
            for j in range(1, len(input_dict[i])+1):
                if random.random() < mix_ratio:
                    # swap the coordinates
                    input_dict[i][j-1], input_dict[i-1][j-1] = input_dict[i-1][j-1], input_dict[i][j-1]
    return input_dict
                    
def mix_up_df(input_df, mix_ratio = 0.3):

    for i in range(1, len(input_df.columns)+1, 2):
        if i == 1:
            continue
        else:
            for j in range(1, len(input_df)+1):
                if random.random() < mix_ratio:
                    # swap the coordinates
                    input_df.iloc[j-1, i-1], input_df.iloc[j-1, i-2] = input_df.iloc[j-1, i-2], input_df.iloc[j-1, i-1]

    return input_df

test_dict = {}
for i in range(1, 11):
    test_dict[i] = coord_generator(tanks_dict[i])


columns = []
for i in range (1, 11):
    columns.append('x' + str(i))
    columns.append('y' + str(i))

test_df = pd.DataFrame(columns = columns)
for i in range(1, 11):
    test_df['x' + str(i)] = [x for x, y in test_dict[i]]
    test_df['y' + str(i)] = [y for x, y in test_dict[i]]

test_df

In [ ]:
display_coords(test_df, im)

In [ ]:
mixed_dict = mix_up(test_dict, mix_ratio = 0.3)
mixed_dict

# turn test_dict into dataframe, with 20 columns
# columns = ['x1', 'y1', 'x2', 'y2', ..., 'x10', 'y10']
# rows = 1000

import pandas as pd



mixed_df = pd.DataFrame(columns = columns)
for i in range(1, 11):
    mixed_df['x' + str(i)] = [x for x, y in mixed_dict[i]]
    mixed_df['y' + str(i)] = [y for x, y in mixed_dict[i]]

mixed_df

In [ ]:
display_coords(mixed_df, im)

In [ ]:
cv2.imshow('Tanks', im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def in_box(input_x, input_y, box):
    x, y, w, h = box
    return x <= input_x <= x + w and y <= input_y <= y + h
    
def in_which_tank(x, y):
    for tank_num, tank_box in tanks_dict.items():
        if in_box(x, y, tank_box):
            return tank_num
    return -1

true_df = pd.DataFrame(columns=test_df.columns)
for idx, row in test_df.iterrows():
    for j in range(1, 11):
        x = row['x' + str(j)]
        y = row['y' + str(j)]
        
        tank = in_which_tank(x, y)

        if tank != -1:
            true_df.loc[idx, 'x' + str(j)] = x
            true_df.loc[idx, 'y' + str(j)] = y     

true_df

In [ ]:
display_coords(true_df, im)

In [ ]:
from Libs.misc import load_threshold

threshold_path = 'Bin/thresholds.json'

speed1, speed2 = load_threshold(threshold_path, 'SPEED_1', 'SPEED_2')

speed1, speed2

In [ ]:
from Libs.misc import get_file_path, load_raw_df, clean_df

file_type = [('Text File', '*.txt')]
default_dir = 'Input'
txt_path = get_file_path(file_type, default_dir)

raw_df, tanks_list = load_raw_df(txt_path)

raw_df, _ = clean_df(raw_df, fill = True)

raw_df.head()


In [ ]:
# export to a csv file
raw_df.to_csv('Output/raw_df_withgap.csv', index = False)

In [ ]:
from Libs.misc import display_coords, get_image

im, _ = get_image(source = 'image')
display_coords(raw_df, im, window_name = "Fish 3", tanks_list = tanks_list)


In [ ]:
from Libs.analyzer import NovelTankTest

nvtt1 = NovelTankTest(raw_df)

summary_dict = nvtt1.basiccal()

summary_dict

In [ ]:
int(15.3)

In [ ]:
from Libs.misc import *

final_df, im, tanks_list = draw_prep()

In [ ]:
draw_trajectories(final_df, im, tanks_list, until = 3000)

In [ ]:
from Libs.misc import *

final_df_1, im_1, tanks_list_1 = draw_prep(mode = 'single')

In [ ]:
import time
import cv2

time_range = [200, 250]

time = time_range[0]
while True:
    print('Time frame: ', time)
    draw_trajectories(final_df_1, im_1, tanks_list_1, until = time, wait = 5)
    time += 1
    cv2.waitKey(1)
    # If "spacebar" is pressed, pause the video
    if cv2.waitKey(1) & 0xFF == ord(' '):
        cv2.waitKey(0)
    if time == time_range[1] or cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
for time in range(200, 250):
    draw_trajectories(final_df_1, im_1, tanks_list_1, until = time, wait = 5)
    # if a key is pressed, pause the video
cv2.waitKey(0)
cv2.destroyAllWindows()

# draw_trajectories(final_df_1, im_1, tanks_list_1, until = 230)

In [ ]:
import numpy as np
import pandas as pd

# create an example dataframe with column X Y
# in X there are some Nan values, along with int values
# in Y there are some Nan values, along with int values

df = pd.DataFrame({'X': [1, 2, 3, np.nan, 5, np.nan, 7, 8, 9, 10],
                     'Y': [1, 2, 3, 4, 5, 6, 7, 8, 9, np.nan]})

# find nan in X
nan_coords = np.where(df['X'].isnull())[0]
len(nan_coords)

In [ ]:
my_array = [1, 2, 3, 4, 7, 8, 9, 15, 16, 17]

# turn it into numpy array
my_array = np.array(my_array)

# split my_array into groups of continous numbers, sep = 1
# output = [[1, 2, 3, 4], [7, 8, 9], [15, 16, 17]]

def split_array(my_array, sep = 1):
    # get the difference between each number
    diff = np.diff(my_array)
    # get the index of the difference that is larger than sep
    split_index = np.where(diff > sep)[0]
    # split the array into groups
    split_array = np.split(my_array, split_index + 1)
    return split_array

split_array(my_array)

In [ ]:
import torch

# test cuda
torch.cuda.is_available()


In [ ]:
from Libs.analyzer import *
import pandas as pd

total_csv_path = r'D:\Code\TowerAssayAnalyzer\Output\arranged_C - Mirror Biting Test - full.csv'

total_df = pd.read_csv(total_csv_path)

# get the first tank
first_tank_df = total_df[['X1', 'Y1']].copy()

NTT_Tank1 = NovelTankTest(first_tank_df)

In [ ]:
NTT_Tank1.distance.variables(magic = False)

## Testing the noveltanktest class

In [ ]:
from Libs.misc import load_raw_df, clean_df
from Libs.executor import *

txt_path = "Input/Zebrafish Tower/A - Novel Tank Test/trajectories.txt"
df, _ = load_raw_df(txt_path)
df, _ = clean_df(df, fill = True)

display_dict = NovelTank_Display(df).rows

for key, value in display_dict.items():
    print(key, value)

## Testing the shoalingtest class

In [ ]:
from Libs.misc import load_raw_df, clean_df
from Libs.executor import *

txt_path = "Input/Zebrafish Tower/A - Novel Tank Test/trajectories.txt"
whole_df, _ = load_raw_df(txt_path)
whole_df, _ = clean_df(whole_df, fill=True)
df1, df2, df3 = whole_df.iloc[:, :2], whole_df.iloc[:, 2:4], whole_df.iloc[:, 4:]

whole_df

In [ ]:
from PIL import Image

im = Image.open('Bin/landing_photo.png')

# crop 200px around the image
crop_px = 400
hor_move = 75
ver_move = -35
im = im.crop((crop_px+hor_move, crop_px+ver_move, im.width - crop_px+hor_move, im.height - crop_px+ver_move))

# save
im.save('Bin/landing_photo_cropped.png')

im

# AUTO ANALYZER

In [1]:
from pathlib import Path
from Libs.batchprocess import MY_DIR


tests = ['Novel Tank', 'Shoaling', 'Mirror Biting', 'Social Interaction', 'Predator Avoidance']
checks = ['novel', 'shoal', 'mirror', 'social', 'predator']
PROJECT_DIR = r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT'
PROJECT_DIR = Path(PROJECT_DIR)

# Get all immediate subdirectories of PROJECT_DIR
subdirectories = [x for x in PROJECT_DIR.glob("*/") if x.is_dir()]
TESTS = {}
for subdirectory in subdirectories:
    if checks[0] in subdirectory.name.lower():
        TESTS[1] = MY_DIR(name=tests[0], dir_path=subdirectory)
    elif checks[1] in subdirectory.name.lower():
        TESTS[2] = MY_DIR(name=tests[1], dir_path=subdirectory, no_gap = True)
    elif checks[2] in subdirectory.name.lower():
        TESTS[3] = MY_DIR(name=tests[2], dir_path=subdirectory)
    elif checks[3] in subdirectory.name.lower():
        TESTS[4] = MY_DIR(name=tests[3], dir_path=subdirectory)
    elif checks[4] in subdirectory.name.lower():
        TESTS[5] = MY_DIR(name=tests[4], dir_path=subdirectory)

# CHECK POINT 1
if len(TESTS) == 5:
    print('TESTS data structure loaded successfully.')


TESTS data structure loaded successfully.


In [2]:
test_num = 1

In [3]:
TESTS[test_num].info()

In Novel Tank Test, we have: 
Batch 1 : [WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/01 - Novel Tank Test/A - Control (1st Batch)'), WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/01 - Novel Tank Test/B - 0.1 ppm (1st Batch)'), WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/01 - Novel Tank Test/C - 10 ppm (1st Batch)')] 



In [4]:
TESTS[test_num].batch[1].info()

In Batch 1, we have: 
Condition A : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch) 
Condition B : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\B - 0.1 ppm (1st Batch) 
Condition C : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\C - 10 ppm (1st Batch) 



In [5]:
TESTS[test_num].batch[1].condition['A'].info()

In condition A, we have: 
Fish group 1 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\1\trajectories.txt 
Fish group 2 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\2\trajectories.txt 
Fish group 3 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\3\trajectories.txt 
Fish group 4 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\4\trajectories.txt 
Fish group 5 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\5\trajectories.txt 
Fish group 6 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\6\trajectories.txt 
Fish group 7 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\7\trajectories.txt 
Fish group 8 : D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\01 - Novel Tank Test\A - Control (1st Batch)\8\trajectories.txt 
Fish g

In [6]:
result_dict_dict = []
for k, v in TESTS[test_num].batch[1].condition['A'].targets.items():
    print(f'Fish group {k:6} : {v.parent.name}/{v.name}')

Fish group 1      : 1/trajectories.txt
Fish group 2      : 2/trajectories.txt
Fish group 3      : 3/trajectories.txt
Fish group 4      : 4/trajectories.txt
Fish group 5      : 5/trajectories.txt
Fish group 6      : 6/trajectories.txt
Fish group 7      : 7/trajectories.txt
Fish group 8      : 8/trajectories.txt
Fish group 9      : 9/trajectories.txt


In [7]:
fish1_path = TESTS[test_num].batch[1].condition['A'].targets['1']

In [8]:
from Libs.executor import noveltank_exec

whole = noveltank_exec(fish1_path, segment = -1)


In [9]:
whole

['whole',
 {0: ['Total Distance', 2481.413168537418, 'cm'],
  1: ['Average Speed', 5.807192063040949, 'cm/s'],
  2: ['Freezing Percentage', 4.231219283875498, '%'],
  3: ['Swimming Percentage', 84.79756611280132, '%'],
  4: ['Average distance to center', 7.6088, 'cm'],
  5: ['Time in top', 21.314237573715246, '%'],
  6: ['Time in bottom', 78.68576242628475, '%'],
  7: ['Time spent in top/bottom ratio', 16.552973130784814, ''],
  8: ['Distance traveled in top', 41074.76550517549, 'cm'],
  9: ['Distance traveled top/bottom ratio', 16.552973130784814, ''],
  10: ['Latency in frames', 2066, 'frames'],
  11: ['Latency in seconds', 41.32, 'seconds'],
  12: ['Number of entries', 54, ''],
  13: ['Average entry', 84.33333333333333, 'seconds']}]

In [10]:
seg = {}
for segment in range(int(420/60)):
    print(f"For segment {segment*5} - {segment*5+1} MIN")
    seg[segment] = noveltank_exec(fish1_path, segment = segment)

For segment 0 - 1 MIN
For segment 5 - 6 MIN
For segment 10 - 11 MIN
For segment 15 - 16 MIN
For segment 20 - 21 MIN
For segment 25 - 26 MIN
For segment 30 - 31 MIN


In [11]:
seg[1]

['5-6 MIN',
 {0: ['Total Distance', 321.1376683369885, 'cm'],
  1: ['Average Speed', 5.354079165338264, 'cm/s'],
  2: ['Freezing Percentage', 5.9686562187395795, '%'],
  3: ['Swimming Percentage', 83.36112037345782, '%'],
  4: ['Average distance to center', 8.9818, 'cm'],
  5: ['Time in top', 19.066666666666666, '%'],
  6: ['Time in bottom', 80.93333333333334, '%'],
  7: ['Time spent in top/bottom ratio', 19.33189537179979, ''],
  8: ['Distance traveled in top', 6208.199804234403, 'cm'],
  9: ['Distance traveled top/bottom ratio', 19.33189537179979, ''],
  10: ['Latency in frames', 37, 'frames'],
  11: ['Latency in seconds', 0.74, 'seconds'],
  12: ['Number of entries', 9, ''],
  13: ['Average entry', 63.55555555555556, 'seconds']}]

In [8]:
import openpyxl

file_path = r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\02 - Shoaling Test (Summary).xlsx'
output_path = r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\merged shoal.xlsx'


def merge_cells(file_path, col_name = 'Shoaling Area', cell_step=3, inplace = True):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(filename=file_path)

    # Get the active worksheet
    worksheet = workbook.active

    # Find the column index for the "Shoaling Area" header
    shoaling_area_col = None
    for col_idx in range(1, worksheet.max_column+1):
        header = worksheet.cell(row=1, column=col_idx).value
        if header and col_name in header:
            shoaling_area_col = col_idx
            break

    if shoaling_area_col is None:
        print("Column not found.")
    else:
        # Merge every next 3 rows of the Shoaling Area column
        for row_idx in range(2, worksheet.max_row+1, cell_step):
            value = worksheet.cell(row=row_idx, column=shoaling_area_col).value
            print(value)
            if value is not None:
                # Merge the current row with the next 2 rows
                worksheet.merge_cells(start_row=row_idx, start_column=shoaling_area_col, end_row=row_idx+2, end_column=shoaling_area_col)
            
            # align the merged cell, horizontal and vertical center
            worksheet.cell(row=row_idx, column=shoaling_area_col).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')
        
    # define output_path
    if inplace == False:
        output_path = file_path[:-5] + '_merged.xlsx'        
    else:
        output_path = file_path    

    # Save the modified workbook
    workbook.save(filename=output_path)

4.5772
8.4746
9.2793


In [20]:
from pathlib import Path

full_path = r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\03 - Mirror Biting Test\B - 0.1 ppm (1st Batch)'

condition_path = Path(full_path)
trajectory_format = "trajectories.txt"
trajectories = condition_path.glob(f'**/{trajectory_format}')
list(trajectories)

[WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/1/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/10/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/2/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/3/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/4/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/5/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/6/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/7/trajectories.txt'),

In [38]:
from pathlib import Path

def sort_paths_by_parent(paths):
    def sort_key(path):
        name_parts = path.parent.name.split('-')
        if len(name_parts) >= 2:
            try:
                primary_key = int(name_parts[0].strip())
            except ValueError:
                primary_key = int(name_parts[0].strip())
            secondary_key = int(name_parts[1].strip())
        else:
            primary_key = int(name_parts[0].strip())
            secondary_key = 0
        return (primary_key, secondary_key)
    
    return sorted(paths, key=sort_key)

full_path = r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT\03 - Mirror Biting Test\B - 0.1 ppm (1st Batch)'
condition_path = Path(full_path)
trajectory_format = "trajectories.txt"
trajectories = condition_path.glob(f'**/{trajectory_format}')

sorted_trajectories = sort_paths_by_parent(trajectories)

sorted_trajectories

[WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/1/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/2/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/2 - 2/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/2 - 5/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/3/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/4/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/5/trajectories.txt'),
 WindowsPath('D:/Code/TowerAssayAnalyzer/Input/OnlyTXT/03 - Mirror Biting Test/B - 0.1 ppm (1st Batch)/6/trajectories

In [7]:
sorted_trajectories = sorted(list(trajectories), key=lambda p: p.parent)
sorted_trajectories

[]

In [16]:
from pathlib import Path
import csv

dir_path = Path(r'D:\Code\TowerAssayAnalyzer\Input\OnlyTXT')

path_list = []

for child_dir in dir_path.rglob('*/'):
    if child_dir.is_dir():
        path_list.append(child_dir.relative_to(dir_path))

path_list = [str(path) for path in path_list]

print(path_list)

# write to .csv file
with open('path_list.csv', 'w') as f:
    for path in path_list:
        f.write("%s %s" % (path, '\n'))

['01 - Novel Tank Test', '02 - Shoaling Test', '03 - Mirror Biting Test', '04 - Social Interaction Test', '05 - Predator Test', '01 - Novel Tank Test\\A - Control (1st Batch)', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)', '01 - Novel Tank Test\\C - 10 ppm (1st Batch)', '01 - Novel Tank Test\\A - Control (1st Batch)\\1', '01 - Novel Tank Test\\A - Control (1st Batch)\\2', '01 - Novel Tank Test\\A - Control (1st Batch)\\3', '01 - Novel Tank Test\\A - Control (1st Batch)\\4', '01 - Novel Tank Test\\A - Control (1st Batch)\\5', '01 - Novel Tank Test\\A - Control (1st Batch)\\6', '01 - Novel Tank Test\\A - Control (1st Batch)\\7', '01 - Novel Tank Test\\A - Control (1st Batch)\\8', '01 - Novel Tank Test\\A - Control (1st Batch)\\9', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)\\1', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)\\10', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)\\2', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)\\3', '01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)\\4

In [ ]:
# load from .csv file
with open('path_list.csv', 'r') as f:
    reader = csv.reader(f)
    path_list = list(reader)

path_list = [path[0].strip() for path in path_list]
path_list

In [ ]:

full_path = [dir_path / Path(path) for path in path_list]

full_path

In [11]:
from pathlib import Path
import csv

dir_path = Path(r'D:\Code\TowerAssayAnalyzer\Input\Test')

structure_csv = 'templates/dir_structure.csv'

with open(structure_csv, 'r') as f:
    reader = csv.reader(f)
    path_list = list(reader)

path_list = [path[0].strip() for path in path_list]
path_list

['01 - Novel Tank Test',
 '02 - Shoaling Test',
 '03 - Mirror Biting Test',
 '04 - Social Interaction Test',
 '05 - Predator Test',
 '01 - Novel Tank Test\\A - Control ({} Batch)',
 '01 - Novel Tank Test\\B - {} {} ({} Batch)',
 '01 - Novel Tank Test\\C - {} {} ({} Batch)',
 '02 - Shoaling Test\\A - Control ({} Batch)',
 '02 - Shoaling Test\\B - {} {} ({} Batch)',
 '02 - Shoaling Test\\C - {} {} ({} Batch)',
 '03 - Mirror Biting Test\\A - Control ({} Batch)',
 '03 - Mirror Biting Test\\B - {} {} ({} Batch)',
 '03 - Mirror Biting Test\\C - {} {} ({} Batch)',
 '04 - Social Interaction Test\\A - Control ({} Batch)',
 '04 - Social Interaction Test\\B - {} {} ({} Batch)',
 '04 - Social Interaction Test\\C - {} {} ({} Batch)',
 '05 - Predator Test\\A - Control ({} Batch)',
 '05 - Predator Test\\B - {} {} ({} Batch)',
 '05 - Predator Test\\C - {} {} ({} Batch)']

In [14]:
path_list[6].format(0.01, 'ppm', '1st')

'01 - Novel Tank Test\\B - 0.01 ppm (1st Batch)'

In [17]:
from pathlib import Path
import json

dir_path = Path(r'D:\Code\TowerAssayAnalyzer\Input\Test')

structure_json = 'templates/dir_structure.json'

with open(structure_json, 'r') as f:
    path_dict = json.load(f)

for path in path_dict['Parent']:
    print(path)

01 - Novel Tank Test
02 - Shoaling Test
03 - Mirror Biting Test
04 - Social Interaction Test
05 - Predator Test


In [18]:
for path in path_dict['Child-Control']:
    print(path.format('1st'))

01 - Novel Tank Test\A - Control (1st Batch)
02 - Shoaling Test\A - Control (1st Batch)
03 - Mirror Biting Test\A - Control (1st Batch)
04 - Social Interaction Test\A - Control (1st Batch)
05 - Predator Test\A - Control (1st Batch)


In [30]:
class PROJECT():

    def __init__(self, name):

        self.name = name
        if self.name == "":
            self.status = False
        else:
            self.status = True

    def give_name(self, name):
        self.name = name
        self.status = True

    def info(self):
        for key, value in self.__dict__.items():
            print(f"{key} : {value}")
        

In [31]:
project = PROJECT("")
project.status

False

In [32]:
project.give_name("test")
project.status

True

In [33]:
project.dir = r'D:\Code\TowerAssayAnalyzer\Input\Test'
project.dir

'D:\\Code\\TowerAssayAnalyzer\\Input\\Test'

In [34]:
project.get_info()

name : test
status : True
dir : D:\Code\TowerAssayAnalyzer\Input\Test


In [36]:
treatment_symbols = [chr(i) for i in range(65, 65+26)]
treatment_symbols

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [43]:
HISTORY_PATH = "History/projects.json"
ORDINALS = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th']
CURRENT_PROJECT = "Melamine"
batch_num = 1
TESTLIST = ['Novel Tank Test', 
                    'Shoaling Test', 
                    'Mirror Biting Test',
                    'Social Interaction Test',
                    'Predator Avoidance Test']

# turn batch_num into ordinal
batch = ORDINALS[int(batch_num)-1]

# change number 2 into B, using mathematically

with open(HISTORY_PATH, "r") as file:
    projects_data = json.load(file)

treatments = projects_data[CURRENT_PROJECT]["TREATMENTS"]

all_paths = {}

all_paths['Parent'] = []
for i, test in enumerate(TESTLIST):
    temp = f"0{i+1} - {test}"
    all_paths['Parent'].append(temp)

tail = f"A - Control ({batch} Batch)"
all_paths['Child-A'] = [f"{parent}\\{tail}" for parent in all_paths['Parent']]

for k, v in treatments.items():
    if k == "Treatment A":
        continue
    char = k.split()[1]
    tail = f"{char} - {v[1]} {v[2]} ({batch} Batch)"
    all_paths[f"Child-{char}"] = [f"{parent}\\{tail}" for parent in all_paths['Parent']]

all_paths

{'Parent': ['01 - Novel Tank Test',
  '02 - Shoaling Test',
  '03 - Mirror Biting Test',
  '04 - Social Interaction Test',
  '05 - Predator Avoidance Test'],
 'Child-A': ['01 - Novel Tank Test\\A - Control (1st Batch)',
  '02 - Shoaling Test\\A - Control (1st Batch)',
  '03 - Mirror Biting Test\\A - Control (1st Batch)',
  '04 - Social Interaction Test\\A - Control (1st Batch)',
  '05 - Predator Avoidance Test\\A - Control (1st Batch)'],
 'Child-B': ['01 - Novel Tank Test\\B - 0.1 ppm (1st Batch)',
  '02 - Shoaling Test\\B - 0.1 ppm (1st Batch)',
  '03 - Mirror Biting Test\\B - 0.1 ppm (1st Batch)',
  '04 - Social Interaction Test\\B - 0.1 ppm (1st Batch)',
  '05 - Predator Avoidance Test\\B - 0.1 ppm (1st Batch)'],
 'Child-C': ['01 - Novel Tank Test\\C - 10.0 ppm (1st Batch)',
  '02 - Shoaling Test\\C - 10.0 ppm (1st Batch)',
  '03 - Mirror Biting Test\\C - 10.0 ppm (1st Batch)',
  '04 - Social Interaction Test\\C - 10.0 ppm (1st Batch)',
  '05 - Predator Avoidance Test\\C - 10.0 ppm 

In [44]:
for v in all_paths['Child-A']:
    print(v)

01 - Novel Tank Test\A - Control (1st Batch)
02 - Shoaling Test\A - Control (1st Batch)
03 - Mirror Biting Test\A - Control (1st Batch)
04 - Social Interaction Test\A - Control (1st Batch)
05 - Predator Avoidance Test\A - Control (1st Batch)


In [56]:
HISTORY_PATH = "History/projects.json"
ORDINALS = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th']
CURRENT_PROJECT = "Melamine"
batch_num = 1
TESTLIST = ['Novel Tank Test', 
                    'Shoaling Test', 
                    'Mirror Biting Test',
                    'Social Interaction Test',
                    'Predator Avoidance Test']

# turn batch_num into ordinal
batch = ORDINALS[int(batch_num)-1]

# change number 2 into B, using mathematically

with open(HISTORY_PATH, "r") as file:
    projects_data = json.load(file)

treatments = projects_data[CURRENT_PROJECT]["TREATMENTS"]

all_paths = {}

all_paths['Parent'] = []
for i, test in enumerate(TESTLIST):
    temp = f"0{i+1} - {test}"
    all_paths['Parent'].append(temp)

for k, v in treatments.items():
    print(v[3])

9
10
10


In [63]:

all_paths['Parent']

normal_parents = []
group_parents = []
for path in all_paths['Parent']:
    if "Shoaling" in path:
        group_parents.append(path)
    else:
        normal_parents.append(path)

normal_parents, group_parents

(['01 - Novel Tank Test',
  '03 - Mirror Biting Test',
  '04 - Social Interaction Test',
  '05 - Predator Avoidance Test'],
 ['02 - Shoaling Test'])

In [ ]:

for k, v in treatments.items():
    char = k.split()[1]
    print(char)
    if char == "A":
        tail = f"A - Control ({batch} Batch)"
    else:
        tail = f"{char} - {v[1]} {v[2]} ({batch} Batch)"

    fish_num = int(v[3])
    all_paths[f"Child-{char}"] = [f"{parent}\\{tail}" for parent in all_paths['Parent']]
    for i in range(1, fish_num+1):
        all_paths[f"Child-{char}"].extend([f"{parent}\\{tail}\\{i}" for parent in all_paths['Parent']])

all_paths